In [1]:
import dash
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import plotly.graph_objs as go

from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash import Input, Output, State

In [2]:
PLOTLY_LOGO = "https://images.plot.ly/logo/new-branding/plotly-logomark.png"

In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

##### Navbar

In [4]:
NAVBAR = dbc.Navbar(
    children=[
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(html.Img(src=PLOTLY_LOGO, height="30px")),
                    dbc.Col(
                        dbc.NavbarBrand("Illinois Cases Analysis", className="ml-2"),
                        style={"marginLeft": 10}
                    ),
                ],
                align="center",
                className="g-0",
            ),
            href="https://github.com/tomfran/legal-texts-information-retrieval",
            style={"margin": 10, "textDecoration": "none"}
        )
    ],
    color="dark",
    dark=True,
    sticky="top",
)

##### Searchbox

In [5]:
SEARCH_BOX = dbc.InputGroup(
    [
        dbc.Button("Search", id="search-button", n_clicks=0),
        dbc.Input(id="search-input", placeholder="cocaine, drug - gun, weapon"),
    ],
    style={"marginTop": 20}
)

In [14]:
@app.callback(
    Output('search-input', 'value'),
    [Input('search-button', 'n_clicks')],
    [State('search-input', 'value')])
def update_output(n_clicks, value):
    if value:
        return f"NEWS: {value.upper()}"

Word Analysis

In [7]:
CONTEXT_GRAPH = dcc.Loading(
    id="loading-similar-context-words",
    children=[dcc.Graph(id="similar-context-graph")],
    type="default",
)
GRAMS_GRAPH = dcc.Loading(
    id="loading-grams", 
    children=[dcc.Graph(id="grams-graph")],
    type="default",
)
WORD_TOPIC_DISTRIBUTION_GRAPH = dcc.Loading(
    id="loading-word-topics", 
    children=[dcc.Graph(id="word-topics-graph")],
    type="default",
)

In [8]:
WORD_CARD = [
    dbc.CardHeader(html.H5("Word analysis")),
    dbc.Alert(
        "Not enough data to render these plots, please adjust the filters",
        id="no-word-data-alert",
        color="warning",
        style={"display": "none"},
    ),
    dbc.CardBody(
        [
            dbc.Row(
                [
                    dbc.Col([
                        dcc.Dropdown(
                            id=f'-dropdown',
                            options=[
                                {'label': 'Actual', 'value': 'Actual'},
                                {'label': 'Predicted', 'value': 'Predicted'}
                            ],
                            value=['Actual', 'Predicted'],
                            multi=True,
                        ), 
                        CONTEXT_GRAPH
                    ]),
                    dbc.Col([dcc.RangeSlider(id="word-time-window-slider"), GRAMS_GRAPH], md=8)
                ]
            ),
            dbc.Row(WORD_TOPIC_DISTRIBUTION_GRAPH)
        ]
    )
]

Topic Analysis

In [9]:
TOPIC_WORDS_GRAPHS = dbc.Row(
    [
        dbc.Col(
            dcc.Loading(
                id="loading-frequencies",
                children=[dcc.Graph(id="frequency_figure")],
                type="default",
            )
        ),
        dbc.Col(
            [
                dcc.Tabs(
                    id="tabs",
                    children=[
                        dcc.Tab(
                            label="Treemap",
                            children=[
                                dcc.Loading(
                                    id="loading-treemap",
                                    children=[dcc.Graph(id="topic-treemap")],
                                    type="default",
                                )
                            ],
                        ),
                        dcc.Tab(
                            label="Wordcloud",
                            children=[
                                dcc.Loading(
                                    id="loading-wordcloud",
                                    children=[
                                        dcc.Graph(id="topic-wordcloud")
                                    ],
                                    type="default",
                                )
                            ],
                        ),
                    ],
                )
            ],
            md=8,
        ),
    ]
)

In [10]:
TOPIC_INFO_GRAPHS = dbc.Row(
    [
        dbc.Col(
            dcc.Loading(
                id="loading-topic-years",
                children=[
                    dcc.Graph(id="topic-years-histogram")
                ],
                type="default",
            )
        ),
        dbc.Col(
            dcc.Loading(
                id="loading-topic-courts",
                children=[
                    dcc.Graph(id="topic-courts-piechart")
                ],
                type="default",
            ),
            md=4
        )
    ]
)

In [11]:
TOPIC_CARD = [
    dbc.CardHeader(html.H5("Topic 5 - Driving Incidents", id="selected_topic_name")),
    dbc.Alert(
        "Not enough data to render these plots, please adjust the filters",
        id="no-topic-data-alert",
        color="warning",
        style={"display": "none"},
    ),
    dbc.CardBody(
        [TOPIC_WORDS_GRAPHS, dcc.RangeSlider(id="topic-time-window-slider"), TOPIC_INFO_GRAPHS]
    )
]

##### Body

In [12]:
BODY = dbc.Container(
    [
        SEARCH_BOX,
        dbc.Card(WORD_CARD, style={"marginTop": 20}),
        dbc.Card(TOPIC_CARD, style={"marginTop": 20, "marginBottom": 30}),
    ],
    className="mt-12",
)

In [13]:
app.layout = html.Div(children=[NAVBAR, BODY])

app.run_server(mode='jupyterlab', dev_tools_ui=True, #debug=True, 
               dev_tools_hot_reload =True, threaded=True)